# Additional SQL - Panda

## The Answers located underneath the relevant code block. 

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
db_config = {xxxxxxxxxxxx"}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                                                             db_config['pwd'], 
                                                                                             db_config['host'], 
                                                                                             db_config['port'], 
                                                                                             db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})   

In [3]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

In [4]:
engine.table_names()

['item', 'rep_sales']

In [5]:
# counting the position played by most of the squad
query = ''' 
              SELECT
              position,
              COUNT(player_id)
              FROM
              squad
              GROUP BY
              position
              ORDER BY
              count DESC;
        '''
queryResult(query)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "squad" does not exist
LINE 6:               squad
                      ^

[SQL:  
              SELECT
              position,
              COUNT(player_id)
              FROM
              squad
              GROUP BY
              position
              ORDER BY
              count DESC;
        ]
(Background on this error at: http://sqlalche.me/e/13/f405)

1) The position that played by most of the squad is defender.

In [ ]:
#summing the games played in all tournamets, joining squad and games played tables and showing the players last name in descending order
query = ''' 
              SELECT
              squad.last_name,
              SUM(games_played.supercup + games_played.cup + games_played.champions + games_played.league) AS total_games_played
              FROM
              games_played
              INNER JOIN squad ON games_played.player_id = squad.player_id
              GROUP BY
              squad.last_name
              ORDER BY
              total_games_played DESC;
        '''
queryResult(query)

2) The players that played the highest number of games this year are: Lopez, Griezmann and De Jong(52, 51 and 51).

In [ ]:
# joining league_stats and squad tables, printing the last name of the player and sum of goals for each player, filtering by players played less then 1000 minutes
query = ''' 
        SELECT
        squad.last_name,
        SUM(league_goals) AS goals
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id
        WHERE
        league_stats.league_minutes < 1000
        GROUP BY
        squad.last_name
        ORDER BY
        goals DESC;
        
        '''
queryResult(query)

3) The top 5 scorers among players that played under 1000 minutes are: Fati(4 goals), Trincão (3 goals), Coutinho(2 goals) and Puig, Roberto, Firpo and Moriba all with 1 goal. 


In [ ]:
#Summing the minutes played by each player in the league, printing last name from the squad table and minutes from league_stats table then ordering by descending minutes
query = ''' 
        SELECT
        squad.last_name,
        SUM(league_minutes) AS league_minutes
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id
        GROUP BY
        squad.last_name
        ORDER BY
        league_minutes DESC;
        
        '''
queryResult(query)

4) The top 5 players by minutes played are: De Jong	(3158 min), Alba (3030 min), Messi (3022 min), Ter Stegen (2790 min) and Griezmann (2619).

In [ ]:
# summing the number of assits across al the tournaments, joining all the tournaments tables with squad table, filtering by forward position. 
query = ''' 
        SELECT
        SUM(league_assists + champions_assists + cup_assists + supercup_assists) AS total_assists
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id 
        INNER JOIN champions_stats ON squad.player_id = champions_stats.player_id
        INNER JOIN cup_stats ON squad.player_id = cup_stats.player_id
        INNER JOIN supercup_stats ON squad.player_id = supercup_stats.player_id
        WHERE
        squad.position = 'fw';
        
        '''
queryResult(query)

5) There were 42 assists made by forwards.

In [ ]:
#dividing number of goals made in all tournaments in the number of minutes played, joining all the tables and ordering by goals per minute in descending order. 
query = ''' 
        SELECT
        squad.last_name,
        (league_goals + cup_goals+supercup_goals+champions_goals)::real / (league_minutes+cup_minutes+supercup_minutes+champions_minutes)::real AS goals_per_minute
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id 
        INNER JOIN champions_stats ON squad.player_id = champions_stats.player_id
        INNER JOIN cup_stats ON squad.player_id = cup_stats.player_id
        INNER JOIN supercup_stats ON squad.player_id = supercup_stats.player_id\
        WHERE
        (league_minutes+cup_minutes+supercup_minutes+champions_minutes) != 0
        GROUP BY
        squad.last_name,
         league_goals,
        cup_goals,
        supercup_goals,
        champions_goals,
        league_minutes,
        cup_minutes,
        supercup_minutes,
        champions_minutes
        ORDER BY
        goals_per_minute DESC;
        
        '''
queryResult(query)

6) The 3 top players goals per minute wise are: Messi(0.009 gpm), Fati(0.008 gpm) and Griezmann (0.005). 

In [ ]:
# Counting the players with 0 goals AND 0 assists in the league
query = ''' 
       SELECT 
       COUNT(player_id)
       FROM
       league_stats
       WHERE
       league_goals = 0 AND league_assists = 0;
        
        '''
queryResult(query)

7) There are 10 players who didn't scored or assisted in this year. 

In [ ]:
# summing yellow cards across players ordering by yellow cards number and minutes played
query = ''' 
        SELECT
        squad.last_name,
        SUM(league_yellow) AS yellow_cards,
        SUM(league_minutes) AS league_minutes
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id
        GROUP BY
        squad.last_name,
        league_minutes
        ORDER BY
        yellow_cards DESC,
        league_minutes DESC;
     
        
        '''
queryResult(query)

8) The players with the most yellow cards in the league are: Alba and Busquets, with 9 cards each.  

In [ ]:
#Summing the number of goals and assists in the league, filtering by only defenders
query = ''' 
        SELECT
        squad.last_name,
        SUM(league_goals + league_assists) AS assists_and_goals
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id
        WHERE
        squad.position = 'df'
        GROUP BY
        squad.last_name
        ORDER BY
        assists_and_goals DESC;
        
        '''
queryResult(query)

9) The defender that contributed the most goals and assists in the league is Alba.

In [ ]:
# changing minutes to hours by division in 60, summing the results in both league and champioship tables, joining tables and filtering by players played more then 50 hours. 
query =   '''
        SELECT
        squad.last_name,
        SUM(league_minutes / 60) AS league_hours,
        SUM(champions_minutes / 60) AS champions_hours,
        SUM(league_minutes / 60 + champions_minutes / 60 ) AS total_hours
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id
        INNER JOIN champions_stats ON squad.player_id = champions_stats.player_id
        WHERE
        (league_minutes / 60 + champions_minutes / 60) > 50 
        GROUP BY
        last_name;
        
        '''
queryResult(query)

10) A total of 5 players played more then 50 hours in the league and in the championship. 